In [2]:
import pandas as pd
import os

# Set folder path
base_path = os.path.join('..', 'Vicon Data', 'Shallow Squat','ID', 'GRF')
raw_path = os.path.join(base_path, 'Raw')
output_path = os.path.join(base_path, 'Continuous')

if not os.path.exists(output_path):
    os.mkdir(os.path.join(base_path, 'Continuous'))
    
# Define force plate - leg mapping
force_plate_map = {'left': 1, 'right': 2}  # 'side': force_plate_number, my left leg was in plate 1 hence left:1, right:2

# Output _grf.mot header
grf_header = [
              'time', 
              'right_ground_force_vx', 'right_ground_force_vy', 'right_ground_force_vz', 
              'right_ground_force_px', 'right_ground_force_py', 'right_ground_force_pz', 
              'right_ground_torque_x', 'right_ground_torque_y', 'right_ground_torque_z',
              'left_ground_force_vx', 'left_ground_force_vy', 'left_ground_force_vz', 
              'left_ground_force_px', 'left_ground_force_py', 'left_ground_force_pz', 
              'left_ground_torque_x', 'left_ground_torque_y', 'left_ground_torque_z'
              ]

In [3]:
# Force & Position

# coordinate mapping (Mokka->OpenSim) and scale factors
force_osim_coordinate_map =  {'X': 'X', 'Y': 'Z', 'Z': 'Y'}
force_osim_scale_factor = [1.0,1.0,-1.0]  # X, Y, Z scale factor for OpenSim
position_osim_scale_factor = [0.001,0.001,-0.001]

# Load force data
force_files = [f for f in os.listdir(raw_path) if f.endswith('force.csv')]
force_dfs =[pd.read_csv(os.path.join(raw_path, force_file), sep=',', skiprows=4, header=[0,1,2]) for force_file in force_files]

# Column name adjustment
for df in force_dfs:
  last_marker = ''
  new_cols = []
  for col in df.columns:
      marker = col[0] if col[0] and not str(col[0]).startswith('Unnamed') else last_marker
      if col[2]:  # X/Y/Z exists
          new_cols.append(f"{marker}_{col[2]}")
      else:
          new_cols.append(marker)
      if col[0] and not str(col[0]).startswith('Unnamed'):
          last_marker = col[0]

  df.columns = new_cols
  df.columns = ['time' if 'Time' in str(c) else c for c in df.columns]

# Convert force and position data to OpenSim .mot format
for df in force_dfs:
    for side, num in force_plate_map.items():
        for i, (osim_axis, mokka_axis) in enumerate(force_osim_coordinate_map.items()):
            df[f"{side}_ground_force_v{osim_axis.lower()}"] = df.pop(f'GRW{num}.F_{mokka_axis}') * force_osim_scale_factor[i]
            df[f"{side}_ground_force_p{osim_axis.lower()}"] = df.pop(f'GRW{num}_{mokka_axis}') * position_osim_scale_factor[i]


In [4]:
# Moment

# coordinate mapping (Mokka->OpenSim) and scale factors
moment_osim_coordinate_map = {'X': 'X', 'Y': 'Z', 'Z': 'Y'}
moment_osim_scale_factor = [-0.001,-0.001,-0.001]  # X, Y, Z scale factor for OpenSim

# Load moment data
moment_files = [f for f in os.listdir(raw_path) if f.endswith('moment.csv')]
moment_dfs = [pd.read_csv(os.path.join(raw_path, moment_file), sep=',', skiprows=[0,1,2,3,4,6], header=[0]) for moment_file in moment_files]

# Convert moment data to OpenSim .mot format
for df in moment_dfs:
    df.columns = ['time' if 'Time' in str(c) else c for c in df.columns]
    for side, num in force_plate_map.items():
        for i, (osim_axis, mokka_axis) in enumerate(moment_osim_coordinate_map.items()):
            df[f"{side}_ground_torque_{osim_axis.lower()}"] = df.pop(f'Moment.M{mokka_axis.lower()}{num+1}') * moment_osim_scale_factor[i]

# Get downsampling factor
with open(os.path.join(raw_path, moment_files[0])) as f:
    for _ in range(2):
        f.readline()  # skip lines
    point_freq = float(f.readline().split(',')[1])
    analog_freq = float(f.readline().split(',')[1])
    factor = int(analog_freq / point_freq)
factor = int(analog_freq / point_freq)

# Downsample moment dataframes (manual nth row selection)
for i, df in enumerate(moment_dfs):
    df_down = df.iloc[::factor].reset_index(drop=True)
    moment_dfs[i] = df_down

In [5]:
# Confirm shapes
print(moment_dfs[0].shape)
print(force_dfs[0].shape)

(1337, 7)
(1337, 13)


In [6]:
# Combine force and moment dataframes
grf_dfs = []
for force_df, moment_df in zip(force_dfs, moment_dfs):
    grf_df = pd.merge_asof(force_df.sort_values('time'), moment_df.sort_values('time'), on='time')
    grf_df = grf_df[[c for c in grf_header if c in grf_df.columns]]
    grf_dfs.append(grf_df)


In [7]:
# Save combined GRF dataframes to _grf.mot files
for i, df in enumerate(grf_dfs):    
    headers = [
    force_files[i].replace('force.csv', 'grf.mot'),
    f"version={1}",
    f"nRows={df.shape[0]}",
    f"nColumns={df.shape[1]}",
    f"inDegrees=yes",
    f"endheader"
    ]
    header_text = '\n'.join(headers + [''])

    out_file = os.path.join(output_path, f"{force_files[i].replace('force.csv', 'grf.mot')}")
    with open(out_file, 'w') as f:
        f.write(header_text)
        df.to_csv(f, sep='\t', index=False, float_format='%.6f')
    print(f"Saved {out_file}")


Saved ..\Vicon Data\Shallow Squat\ID\GRF\Continuous\Hanif_Shallow_2_5_grf.mot
Saved ..\Vicon Data\Shallow Squat\ID\GRF\Continuous\Hanif_Shallow_5_0_grf.mot
Saved ..\Vicon Data\Shallow Squat\ID\GRF\Continuous\Hanif_Shallow_7_5_grf.mot
